In [0]:
import os
import pathlib
import random
import tensorflow as tf
import tensorflow_hub as tfhub
#tf.enable_eager_execution()
from IPython.core.debugger import Tracer

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_steps = 150 # 500 Modelo Train
summary_steps = 50 # 100 Modelo Train
steps_per_epoch = 100 # 100 Modelo Train
log_step = 10 # 10 Modelo Train
train_batch = 5 # 5 512 Funciones de input train
eval_batch = 5 # 5 500 Funciones de input eval
eval_delay_secs = 10 # 10 Modelo Eval
throttle_secs = 30 # 30 Modelo Eval
CLASSES = 2

In [0]:
tf.logging.info('TF Version {}'.format(tf.__version__))
tf.logging.info('GPU Available {}'.format(tf.test.is_gpu_available()))
if 'TF_CONFIG' in os.environ:
    tf.logging.info('TF_CONFIG: {}'.format(os.environ["TF_CONFIG"]))

**Rutas**

In [0]:
# Modelo general
path_model = 'model/' #<--- en mac 'model/' en colab '/model/'
os.makedirs(os.path.join(path_model), exist_ok=True)

# Modelo entrenado
path_trained = os.path.join(path_model, 'trained')
os.makedirs(path_trained, exist_ok=True)

# Modelo importado
TFHUB_CACHE_DIR = os.path.join(path_model, 'TFHub')
os.environ['TFHUB_CACHE_DIR'] = TFHUB_CACHE_DIR
#os.makedirs(TFHUB_CACHE_DIR, exist_ok=True)

# Data importada
path_data = os.path.join(path_model, 'data')
os.makedirs(path_data, exist_ok=True)
#url_data = 'http://files.fast.ai/data/dogscats.zip'
#if not os.path.isdir(path_data): tf.keras.utils.get_file(fname = os.path.basename(url_data), origin = url_data, cache_subdir=path_model, extract=True)


# Modelo importado
**Importacion**

In [0]:
if not os.path.isdir(TFHUB_CACHE_DIR):
  tfh_module = tfhub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2")
  #tfh_module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_050_192/feature_vector/2")
  #tfh_module = hub.Module("https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/1")
  #tfh_module = tfhub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2")
else:
  tfh_module = tfhub.Module(os.path.join(TFHUB_CACHE_DIR, 'adfe0cf8d843e3588bfb9602e32a718b12212904'))

**Variables derivadas**

In [0]:
IMAGE_SIZE = tfhub.get_expected_image_size(tfh_module)

# Datos


**Directorios de datos**

In [0]:
train_path = os.path.join(path_data, 'train')
eval_path = os.path.join(path_data, 'eval')
pred_path = os.path.join(path_data, 'pred/valid')

---
For this output, you can connect additional layers according to the problem you want to solve. For example, if the problem is to classify images into 10 categories, you may want to continue fully connected-layer with 10 nodes, and apply the softmax function to have the probabilities.


logits = tf.layers.dense(inputs=outputs, units=10)
predictions = {
    "classes": tf.argmax(input=logits, axis=1),
    "probabilities": tf.nn.softmax(logits)
}

---
train_input_fn = tf.data.Dataset.list_files(train_files)

---
from IPython.core.debugger import Tracer; Tracer()() 
n, c, q

# Dataset

In [0]:
def load_and_preprocess_image(path):
  
  image = tf.read_file(path)
  image = tf.image.decode_jpeg(image, channels=3) #<-------
  image = tf.image.resize(image, IMAGE_SIZE) #<-------
  image /= 255.0  # normalize to [0,1] range #<-------
  return image

In [0]:
def read_dataset(filename, mode, batch_size):  

  data_root = pathlib.Path(filename)#<------- filename
  all_image_paths = list(data_root.glob('*/*'))
  all_image_paths = [str(path) for path in all_image_paths]
  random.shuffle(all_image_paths) #<---- super importante para el desempeno del modelo

  #Como mejorar estas lineas
  label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
  label_to_index = dict((name, index) for index, name in enumerate(label_names))
  all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
  paths_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
  image_ds = paths_ds.map(load_and_preprocess_image)
  label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
  
  ds = tf.data.Dataset.zip((image_ds, label_ds)) #<-------
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    num_epochs = None #Loop indefinidamente
    ds = ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=batch_size*10)) # video 7:10 https://www.youtube.com/watch?v=uIcqeP7MFH0
  else:
    num_epochs = 1 #end-of-input after this
  
  ds = ds.batch(batch_size).repeat(num_epochs).prefetch(AUTOTUNE) # video 7:48 .repeat(num_epochs) 
  return ds

## Funciones de entrada

In [0]:
def train_input_fn():
  return read_dataset(filename = train_path,#<-------
                      batch_size = train_batch, 
                      mode = tf.estimator.ModeKeys.TRAIN)

def eval_input_fn():
  return read_dataset(filename = eval_path,#<-------
                      batch_size = eval_batch, 
                      mode = tf.estimator.ModeKeys.EVAL)

def pred_input_fn():
  return read_dataset(filename = pred_path,#<------- 
                      batch_size = train_batch,  #<---- No va en prediccion
                      mode = tf.estimator.ModeKeys.PREDICT)

def serving_input_fn():
  json = {'images' : tf.placeholder(tf.string, [None])} #<--- puede ser shape=(None, 224, 224, 3)

  def decode(jpeg):
    pixels = tf.image.decode_jpeg(jpeg, channels=3)
    return pixels
  
  pics = tf.map_fn(decode, json['images'], dtype=tf.float32)
  features = {'images': pics}
  return tf.estimator.export.ServingInputReceiver(features, json)

# Modelo
**Configuracion del modelo importado**

In [0]:
def model_fn(features, labels, mode, params):
  tfh_module=tfhub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2")
  transformed_features = tfh_module(features)
  logits = tf.layers.dense(transformed_features, CLASSES) #<-------
  probabilities = tf.nn.softmax(logits)
  
  if (mode != tf.estimator.ModeKeys.PREDICT):
    one_hot_labels = tf.one_hot(labels, CLASSES) #<-------
    loss = tf.losses.softmax_cross_entropy(one_hot_labels, logits)
    optimizer = tf.train.AdamOptimizer() 
    train_op = tf.contrib.training.create_train_op(loss, optimizer)
    accuracy = tf.metrics.accuracy(labels, tf.argmax(probabilities, axis=-1))
    metrics = {'acc':accuracy}
  else:
    # None of these can be computed in prediction mode because labels are not available
    loss = optimizer = train_op = metrics = None
  
  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=train_op,
      # Nice: in estimator, you can return computed results that are not part of model training (here: class)
      predictions={'proba': probabilities, 'class': tf.argmax(probabilities, axis=-1)},
      eval_metric_ops=metrics
  )

# Entrenamiento

In [0]:
def train_and_evaluate(): #args
  # Estimador del modelo a usar
  run_config = tf.estimator.RunConfig(
    model_dir = path_trained, #args['output_dir'],  Ouput directory for checkpoint
    save_summary_steps = summary_steps, 
    save_checkpoints_steps = steps_per_epoch, #, save_checkpoints_step = 100
    log_step_count_steps=log_step)

  # Define los aspectos del modelo # ojo aqui esta el transfer learning
  model = tf.estimator.Estimator(
    model_fn = model_fn,
    config = run_config)

  # Define los aspectos del entrenamiento y la entrada de datos
  train_spec = tf.estimator.TrainSpec(
    input_fn = train_input_fn, 
    max_steps = train_steps) #args['train_steps']

  # Define los aspectos del uso en produccion con ML Engine
  export_latest = tf.estimator.LatestExporter(
    'exporter', #folder to export
    serving_input_receiver_fn = serving_input_fn)

  # Define los aspectos de la evaluacion, cada cuanto se graba para tensorboard y la entrada de datos
  eval_spec = tf.estimator.EvalSpec(
    input_fn = eval_input_fn,
    steps = None,
    start_delay_secs = eval_delay_secs, # args['eval_delay_secs'] start evaluating after N seconds
    throttle_secs = throttle_secs, #, args['throttle_secs'] evaluate every N seconds
    exporters = export_latest)

  # Ejecuta el modelo
  out = tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
  print(out)
  return model

# Prediccion

In [0]:
import shutil
if os.path.isdir(path_trained):
  shutil.rmtree(path_trained, ignore_errors=False, onerror=None)

trained_model = train_and_evaluate()
print(trained_model)

In [0]:
predictions = trained_model.predict(input_fn = pred_input_fn)
for items in predictions:
    print(items)